In [ ]:
!pip install opencv-contrib-python-headless

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import files
from google.colab.patches import cv2_imshow
# Step 1: Define Dark Channel Prior Function
def dark_channel_prior(image, patch_size=25):
    dark_channel = cv2.min(cv2.min(image[:, :, 0], image[:, :, 1]), image[:, :, 2])
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (patch_size, patch_size))
    dark_channel = cv2.erode(dark_channel, kernel)
    return dark_channel

# Step 2: Estimate Atmospheric Light
def estimate_atmospheric_light(image, dark_channel):
    num_pixels = image.shape[0] * image.shape[1]
    num_brightest_pixels = int(max(num_pixels * 0.001, 1))
    brightest_pixels_indices = np.argpartition(dark_channel.ravel(), -num_brightest_pixels)[-num_brightest_pixels:]
    atmospheric_light = np.mean(image.reshape(-1, 3)[brightest_pixels_indices], axis=0)
    return atmospheric_light

# Step 3: Estimate Transmission Map
def estimate_transmission(image, atmospheric_light, omega=0.85, patch_size=25):
    normalized_img = image / atmospheric_light
    transmission = 1 - omega * dark_channel_prior(normalized_img, patch_size)
    return transmission
def apply_guided_filter(image, transmission_map, radius=60, epsilon=1e-3):
    gray_image = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
    gray_image = gray_image.astype(np.float32) / 255.0
    # Create a Guided Filter object
    guided_filter = cv2.ximgproc.createGuidedFilter(gray_image.astype(np.uint8), radius, epsilon)
    # Apply the filter
    refined_transmission = guided_filter.filter(transmission_map.astype(np.float32))
    return refined_transmission



# Step 5: Recover the Scene Radiance
def recover_image(image, transmission, atmospheric_light, t_min=0.2):
    atmospheric_light = atmospheric_light.reshape((1, 1, 3))
    transmission = np.repeat(transmission[:, :, np.newaxis], 3, axis=2)
    transmission = np.maximum(transmission, t_min)
    recovered_img = (image - atmospheric_light) / transmission + atmospheric_light
    recovered_img = np.clip(recovered_img, 0, 1)
    return (recovered_img * 255).astype(np.uint8)

# Step 6: Upload and Read Video
uploaded = files.upload()
input_video_path = list(uploaded.keys())[0]

# Create directory to save individual frames
output_frames_dir = "dehazed_frames"
os.makedirs(output_frames_dir, exist_ok=True)

# Open the video file
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Step 7: Process Video Frame by Frame and Save Individual Frames
output_video_path = 'final_dehazed_output.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB (OpenCV reads in BGR by default)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_rgb = frame_rgb.astype(np.float32) / 255.0

    # Apply dehazing
    dark_channel = dark_channel_prior(frame_rgb)
    atmospheric_light = estimate_atmospheric_light(frame_rgb, dark_channel)
    transmission_map = estimate_transmission(frame_rgb, atmospheric_light)
    refined_transmission_map = apply_guided_filter(frame_rgb, transmission_map)
    dehazed_frame = recover_image(frame_rgb, refined_transmission_map, atmospheric_light)

    # Convert the dehazed frame back to BGR for saving
    dehazed_frame_bgr = cv2.cvtColor(dehazed_frame, cv2.COLOR_RGB2BGR)

    # Write the dehazed frame to the output video
    out.write(dehazed_frame_bgr)

cap.release()
out.release()

# Step 8: Create a Combined Side-by-Side Video
def create_combined_video(original_video_path, dehazed_video_path, output_combined_path):
    cap_original = cv2.VideoCapture(original_video_path)
    cap_dehazed = cv2.VideoCapture(dehazed_video_path)

    # Get video properties
    fps = cap_original.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap_original.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap_original.get(cv2.CAP_PROP_FRAME_HEIGHT))
    combined_width = frame_width * 2

    # Video writer for the combined video
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out_combined = cv2.VideoWriter(output_combined_path, fourcc, fps, (combined_width, frame_height))

    while cap_original.isOpened() and cap_dehazed.isOpened():
        ret_original, frame_original = cap_original.read()
        ret_dehazed, frame_dehazed = cap_dehazed.read()

        if not ret_original or not ret_dehazed:
            break

        # Resize dehazed frame to match the original frame size if needed
        frame_dehazed_resized = cv2.resize(frame_dehazed, (frame_width, frame_height))

        # Concatenate original and dehazed frames horizontally
        combined_frame = cv2.hconcat([frame_original, frame_dehazed_resized])

        # Write the combined frame to the output video
        out_combined.write(combined_frame)

    cap_original.release()
    cap_dehazed.release()
    out_combined.release()

    print(f"Combined video saved as '{output_combined_path}'. You can download it now.")

# Create and save the combined video
combined_output_video_path = 'combined_side_by_side_output.avi'
create_combined_video(input_video_path, output_video_path, combined_output_video_path)

# Display a preview of the combined video (first frame)
cap_combined = cv2.VideoCapture(combined_output_video_path)
ret, frame = cap_combined.read()
if ret:
    cv2_imshow(frame)

cap_combined.release()
!apt-get install ffmpeg
import os
from IPython.display import HTML, display
from base64 import b64encode

def play_video(combined_output_video_path):
    """
    Plays a video in the Jupyter Notebook.

    Args:
        combined_output_video_path (str): The path to the video file.

    Returns:
        IPython.display.HTML: HTML code for embedding the video player.
    """
    if not os.path.exists(combined_output_video_path):
        return HTML("<p>Error: Video file not found.</p>")

    # Convert the video to a widely compatible format (MP4 with H.264 video and AAC audio)
    # Use a temporary file for the converted video to avoid overwriting the original
    converted_video_path = os.path.splitext(combined_output_video_path)[0] + '_converted.mp4'
    !ffmpeg -y -i "$combined_output_video_path" -c:v libx264 -preset medium -crf 23 -c:a aac -strict experimental "$converted_video_path"

    # Display the video using HTML5 video tag with controls
    video_data = open(converted_video_path, "rb").read()
    encoded_video = b64encode(video_data).decode()

    return HTML("""
        <video width="640" height="480" controls>
            <source src="data:video/mp4;base64,{0}" type="video/mp4">
            Your browser does not support the video tag.
        </video>
    """.format(encoded_video.replace("\n", "")))

# Call the function to play the video
display(play_video(combined_output_video_path))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 MB 12.6 MB/s eta 0:00:00
